## Procesamiento de Lenguage Natural

### Taller #8: Modelado de temas
#### Rubio de Jesus Vasquez 

In [3]:
import re
import pandas as pd 
from pprint import pprint

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

import pyLDAvis.gensim
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [4]:
# Cargar datos
path = 'C:/Personal/SemestreII/NLP/Talleres/Taller7/reviews_vidjew_es.csv'
data = pd.read_csv(path)
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games


#### Punto 1: Hacer pre-procesamiento del texto

In [6]:
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords]
    return texto

data['pp'] = data['review_body'].apply(lambda texto: pre_procesado(texto))

data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,Pre-Processed,pp
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]","[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw...","[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl...","[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble...","[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg...","[cable, funciona, bien, podria, ser, mas, larg..."


###  Punto 2: Modelo de LDA

In [8]:
# Crear una representación de los documentos en forma de diccionario
dictionary = Dictionary(data['pp'].values)

# Filtrar palabras muy frecuentes o infrecuentes
dictionary.filter_extremes(no_below=5, no_above=0.5)

corpus = [dictionary.doc2bow(text) for text in data['pp'].values]

# Train the topic model
model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=7, passes=50)

In [9]:
# Imprimir temas
model.print_topics(num_words=15)

[(0,
  '0.067*"juego" + 0.033*"si" + 0.026*"bien" + 0.021*"jugar" + 0.017*"ps" + 0.016*"solo" + 0.016*"juegos" + 0.014*"original" + 0.013*"caja" + 0.011*"aunque" + 0.011*"mejor" + 0.010*"va" + 0.010*"caso" + 0.010*"español" + 0.010*"pena"'),
 (1,
  '0.041*"bien" + 0.035*"producto" + 0.033*"calidad" + 0.025*"buen" + 0.023*"precio" + 0.022*"regalo" + 0.020*"bonita" + 0.020*"buena" + 0.016*"llegado" + 0.015*"caja" + 0.014*"queda" + 0.013*"collar" + 0.012*"plástico" + 0.012*"mismo" + 0.011*"cumple"'),
 (2,
  '0.031*"perfecto" + 0.031*"llegó" + 0.024*"producto" + 0.023*"día" + 0.020*"llego" + 0.020*"si" + 0.015*"nunca" + 0.014*"comprar" + 0.014*"puedo" + 0.014*"pulsera" + 0.013*"roto" + 0.012*"pedí" + 0.012*"meses" + 0.012*"llega" + 0.012*"envío"'),
 (3,
  '0.047*"bonito" + 0.032*"foto" + 0.029*"bien" + 0.029*"tamaño" + 0.027*"bonitos" + 0.023*"grande" + 0.020*"grandes" + 0.019*"queda" + 0.018*"tal" + 0.017*"quedan" + 0.016*"mal" + 0.016*"así" + 0.016*"pequeño" + 0.016*"regalo" + 0.015*"dem

 ### Punto 3: Visualización de LDA

In [10]:
lda_display = pyLDAvis.gensim.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)
# pyLDAvis.save_html(lda_display, 'lda.html')